In [1]:
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
import numpy as np
import os
import pandas as pd


# テキストファイルをnumpy配列に読み込み
all_data = np.loadtxt("claim_toiawase_data_python.txt")
#numpy配列をDataFrame型に変換(でーたを見やすくするため)
all_data_pd = pd.DataFrame(all_data)
#DataFrame型の列名を設定
all_data_pd.columns = ["seq", "Imp", "Imp_val", "m_sum_neg", "c_neg", "m_sum_all", "m_sum_nzr", "Mag", "c_nzr", "c_all", "m_max", "s_min", "m_sdv_neg", 
                   "s_sdv_neg", "m_avg_neg", "s_avg_neg", "s_sdv_nzr", "m_sdv_all", "s_sdv_all", "s_avg_nzr", "m_avg_nzr", "s_avg_all", "Sc",
                   "m_avg_all", "m_sum_pos", "c_pos", "m_min", "s_max", "m_sdv_pos", "s_sdv_pos", "m_avg_pos", "s_avg_pos", "m_sdv_nzr"]
print(all_data_pd)   #ImpとImp_valは同じ

# 特徴量の部分のみall_Xに読み込み
all_X = all_data[:9813,3:33]
#ラベル部分をall_yに読み込み
all_y = all_data[:9813, 2]
print(all_X.shape)
print(all_y)
# print(type(all_X))

         seq  Imp  Imp_val  m_sum_neg  c_neg  m_sum_all  m_sum_nzr  Mag  \
0        1.0  1.0      1.0        1.0    3.0        1.1        1.1  1.2   
1        2.0  1.0      1.0        0.5    1.0        0.5        0.5  0.5   
2        3.0  1.0      1.0        2.1    3.0        2.1        2.1  2.2   
3        4.0  1.0      1.0        1.9    3.0        1.9        1.9  2.0   
4        5.0  1.0      1.0        0.4    1.0        0.5        0.5  0.5   
5        6.0  1.0      1.0        0.5    1.0        0.5        0.5  0.5   
6        7.0  1.0      1.0        3.0    6.0        3.0        3.0  3.2   
7        8.0  1.0      1.0        1.7    3.0        2.6        2.6  2.8   
8        9.0  1.0      1.0        1.1    3.0        1.3        1.3  1.5   
9       10.0  1.0      1.0        1.3    2.0        1.3        1.3  1.3   
10      11.0  1.0      1.0        3.8    5.0        3.8        3.8  4.1   
11      12.0  1.0      1.0        1.0    2.0        1.0        1.0  1.0   
12      13.0  1.0      1.

In [3]:
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt

#アップサンプリング前のクラス1のデータ数とクラス0のデータ数の出力
print("Number of class 1 samples before:", all_y[all_y == 1].shape[0])
print("Number of class 0 samples before:", all_y[all_y == 0].shape[0])
# print(all_X[all_y == 1])
# print(all_X[all_y == 0])

#クラスの1のサンプルの個数がクラス0と同じになるまで新しいサンプルを復元抽出
X_upsampled, y_upsampled = resample(all_X[all_y == 1],
                                    all_y[all_y == 1],
                                    replace = True,
                                    n_samples = all_X[all_y == 0].shape[0],
                                    random_state = 123)
# アップサンプリング後のクラス１のデータ数とデータの中身の出力
print("Number of class 1 samples after:", X_upsampled.shape[0])
# print(X_upsampled)
# 元のクラス０のサンプルにアップサンプリングしたクラス０のサブセットを結合
X_bal = np.vstack((all_X[all_y == 0], X_upsampled))  #新しい特徴量集合
y_bal = np.hstack((all_y[all_y == 0], y_upsampled))  #新しいラベル集合
print(X_bal.shape, y_bal.shape)

print(X_bal)
X_bal = X_bal[:16794,0:1]
print(X_bal)
# 新しい特徴量集合とラベル集合をそれぞれ7:3の割合で訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X_bal, y_bal, test_size=0.3, random_state=0, stratify=y_bal )

# ニューラルネットワークのオブジェクト生成
nn = MLPClassifier(hidden_layer_sizes = (10, ),     #素子数の数（層の数も指定可能）
                   activation = "logistic",             #活性化関数の指定
                   solver = "sgd",                 #重みの最適化の解法指定
                   alpha = 0.0001,                  #l2正則化のパラメータ指定
                   batch_size = 9813,             #ミニバッチ数のサイズ指定
                   learning_rate = "constant",          #学習率タイプの指定
                   learning_rate_init = 0.001,     #学習率の初期値設定
                   max_iter = 1000000,            #繰り返し回数の指定
                   shuffle = True,             #繰り返し毎にサンプルをシャッフルするかしないか
                   random_state = 1,           #シードの設定
                   tol = 0,
                   verbose = True,                #進捗状況を出力するか否か
                   momentum = 0.5,               #慣性項の指定
                   nesterovs_momentum = True,     #ネストロフのモーメンタムを利用するかどうか
                   early_stopping = False)         #早期停止するかどうか


nn.fit(X_train, y_train)
print(nn.predict(X_test))

Number of class 1 samples before: 1416
Number of class 0 samples before: 8397
Number of class 1 samples after: 8397
(16794, 30) (16794,)
[[0.4  1.   0.4  ... 0.   0.   0.93]
 [0.3  1.   0.3  ... 0.   0.   1.01]
 [0.4  1.   0.4  ... 0.   0.   1.08]
 ...
 [1.7  3.   1.7  ... 0.   0.   0.75]
 [1.3  2.   1.3  ... 0.   0.   0.92]
 [0.8  2.   2.1  ... 0.33 0.33 0.53]]
[[0.4]
 [0.3]
 [0.4]
 ...
 [1.7]
 [1.3]
 [0.8]]
Iteration 1, loss = 0.68840149
Iteration 2, loss = 0.68838326
Iteration 3, loss = 0.68836234
Iteration 4, loss = 0.68834040
Iteration 5, loss = 0.68831799
Iteration 6, loss = 0.68829598
Iteration 7, loss = 0.68827417
Iteration 8, loss = 0.68825208
Iteration 9, loss = 0.68822980
Iteration 10, loss = 0.68820799
Iteration 11, loss = 0.68818630
Iteration 12, loss = 0.68816430
Iteration 13, loss = 0.68814216
Iteration 14, loss = 0.68811976
Iteration 15, loss = 0.68809732
Iteration 16, loss = 0.68807556
Iteration 17, loss = 0.68805392
Iteration 18, loss = 0.68803185
Iteration 19, loss =

Iteration 242, loss = 0.68324549
Iteration 243, loss = 0.68322489
Iteration 244, loss = 0.68320409
Iteration 245, loss = 0.68318325
Iteration 246, loss = 0.68316238
Iteration 247, loss = 0.68314203
Iteration 248, loss = 0.68312170
Iteration 249, loss = 0.68310104
Iteration 250, loss = 0.68308023
Iteration 251, loss = 0.68305933
Iteration 252, loss = 0.68303808
Iteration 253, loss = 0.68301699
Iteration 254, loss = 0.68299641
Iteration 255, loss = 0.68297620
Iteration 256, loss = 0.68295540
Iteration 257, loss = 0.68293461
Iteration 258, loss = 0.68291396
Iteration 259, loss = 0.68289335
Iteration 260, loss = 0.68287237
Iteration 261, loss = 0.68285167
Iteration 262, loss = 0.68283103
Iteration 263, loss = 0.68281035
Iteration 264, loss = 0.68278960
Iteration 265, loss = 0.68276884
Iteration 266, loss = 0.68274799
Iteration 267, loss = 0.68272704
Iteration 268, loss = 0.68270637
Iteration 269, loss = 0.68268605
Iteration 270, loss = 0.68266552
Iteration 271, loss = 0.68264499
Iteration 

Iteration 494, loss = 0.67810291
Iteration 495, loss = 0.67808295
Iteration 496, loss = 0.67806309
Iteration 497, loss = 0.67804301
Iteration 498, loss = 0.67802302
Iteration 499, loss = 0.67800304
Iteration 500, loss = 0.67798297
Iteration 501, loss = 0.67796310
Iteration 502, loss = 0.67794314
Iteration 503, loss = 0.67792316
Iteration 504, loss = 0.67790319
Iteration 505, loss = 0.67788336
Iteration 506, loss = 0.67786323
Iteration 507, loss = 0.67784308
Iteration 508, loss = 0.67782299
Iteration 509, loss = 0.67780289
Iteration 510, loss = 0.67778293
Iteration 511, loss = 0.67776300
Iteration 512, loss = 0.67774296
Iteration 513, loss = 0.67772326
Iteration 514, loss = 0.67770308
Iteration 515, loss = 0.67768294
Iteration 516, loss = 0.67766323
Iteration 517, loss = 0.67764368
Iteration 518, loss = 0.67762386
Iteration 519, loss = 0.67760363
Iteration 520, loss = 0.67758366
Iteration 521, loss = 0.67756376
Iteration 522, loss = 0.67754378
Iteration 523, loss = 0.67752383
Iteration 

Iteration 743, loss = 0.67317434
Iteration 744, loss = 0.67315469
Iteration 745, loss = 0.67313490
Iteration 746, loss = 0.67311523
Iteration 747, loss = 0.67309562
Iteration 748, loss = 0.67307596
Iteration 749, loss = 0.67305634
Iteration 750, loss = 0.67303692
Iteration 751, loss = 0.67301712
Iteration 752, loss = 0.67299750
Iteration 753, loss = 0.67297821
Iteration 754, loss = 0.67295881
Iteration 755, loss = 0.67293942
Iteration 756, loss = 0.67292006
Iteration 757, loss = 0.67290043
Iteration 758, loss = 0.67288086
Iteration 759, loss = 0.67286175
Iteration 760, loss = 0.67284252
Iteration 761, loss = 0.67282304
Iteration 762, loss = 0.67280361
Iteration 763, loss = 0.67278414
Iteration 764, loss = 0.67276468
Iteration 765, loss = 0.67274501
Iteration 766, loss = 0.67272548
Iteration 767, loss = 0.67270616
Iteration 768, loss = 0.67268695
Iteration 769, loss = 0.67266762
Iteration 770, loss = 0.67264809
Iteration 771, loss = 0.67262870
Iteration 772, loss = 0.67260925
Iteration 

Iteration 1000, loss = 0.66820761
Iteration 1001, loss = 0.66818829
Iteration 1002, loss = 0.66816892
Iteration 1003, loss = 0.66814939
Iteration 1004, loss = 0.66812986
Iteration 1005, loss = 0.66811039
Iteration 1006, loss = 0.66809084
Iteration 1007, loss = 0.66807170
Iteration 1008, loss = 0.66805262
Iteration 1009, loss = 0.66803339
Iteration 1010, loss = 0.66801392
Iteration 1011, loss = 0.66799461
Iteration 1012, loss = 0.66797529
Iteration 1013, loss = 0.66795615
Iteration 1014, loss = 0.66793675
Iteration 1015, loss = 0.66791722
Iteration 1016, loss = 0.66789757
Iteration 1017, loss = 0.66787818
Iteration 1018, loss = 0.66785901
Iteration 1019, loss = 0.66784018
Iteration 1020, loss = 0.66782155
Iteration 1021, loss = 0.66780265
Iteration 1022, loss = 0.66778380
Iteration 1023, loss = 0.66776470
Iteration 1024, loss = 0.66774544
Iteration 1025, loss = 0.66772621
Iteration 1026, loss = 0.66770736
Iteration 1027, loss = 0.66768843
Iteration 1028, loss = 0.66766941
Iteration 1029

...

Iteration 30883, loss = 0.52821031
Iteration 30884, loss = 0.52821030
Iteration 30885, loss = 0.52821026
Iteration 30886, loss = 0.52821023
Iteration 30887, loss = 0.52821019
Iteration 30888, loss = 0.52821017
Iteration 30889, loss = 0.52821013
Iteration 30890, loss = 0.52821008
Iteration 30891, loss = 0.52821004
Iteration 30892, loss = 0.52821000
Iteration 30893, loss = 0.52820996
Iteration 30894, loss = 0.52820994
Iteration 30895, loss = 0.52820995
Iteration 30896, loss = 0.52820992
Iteration 30897, loss = 0.52820983
Iteration 30898, loss = 0.52820981
Iteration 30899, loss = 0.52820982
Iteration 30900, loss = 0.52820974
Iteration 30901, loss = 0.52820973
Iteration 30902, loss = 0.52820972
Iteration 30903, loss = 0.52820970
Iteration 30904, loss = 0.52820960
Iteration 30905, loss = 0.52820959
Iteration 30906, loss = 0.52820951
Iteration 30907, loss = 0.52820948
Iteration 30908, loss = 0.52820944
Iteration 30909, loss = 0.52820940
Iteration 30910, loss = 0.52820942
Iteration 30911, los

Iteration 31129, loss = 0.52820170
Iteration 31130, loss = 0.52820166
Iteration 31131, loss = 0.52820163
Iteration 31132, loss = 0.52820162
Iteration 31133, loss = 0.52820159
Iteration 31134, loss = 0.52820154
Iteration 31135, loss = 0.52820152
Iteration 31136, loss = 0.52820149
Iteration 31137, loss = 0.52820147
Iteration 31138, loss = 0.52820144
Iteration 31139, loss = 0.52820140
Iteration 31140, loss = 0.52820141
Iteration 31141, loss = 0.52820140
Iteration 31142, loss = 0.52820132
Iteration 31143, loss = 0.52820133
Iteration 31144, loss = 0.52820130
Iteration 31145, loss = 0.52820125
Iteration 31146, loss = 0.52820117
Iteration 31147, loss = 0.52820122
Iteration 31148, loss = 0.52820109
Iteration 31149, loss = 0.52820112
Iteration 31150, loss = 0.52820108
Iteration 31151, loss = 0.52820105
Iteration 31152, loss = 0.52820102
Iteration 31153, loss = 0.52820101
Iteration 31154, loss = 0.52820095
Iteration 31155, loss = 0.52820092
Iteration 31156, loss = 0.52820091
Iteration 31157, los

Iteration 31364, loss = 0.52819433
Iteration 31365, loss = 0.52819430
Iteration 31366, loss = 0.52819420
Iteration 31367, loss = 0.52819416
Iteration 31368, loss = 0.52819411
Iteration 31369, loss = 0.52819405
Iteration 31370, loss = 0.52819400
Iteration 31371, loss = 0.52819399
Iteration 31372, loss = 0.52819401
Iteration 31373, loss = 0.52819389
Iteration 31374, loss = 0.52819386
Iteration 31375, loss = 0.52819385
Iteration 31376, loss = 0.52819389
Iteration 31377, loss = 0.52819385
Iteration 31378, loss = 0.52819395
Iteration 31379, loss = 0.52819379
Iteration 31380, loss = 0.52819387
Iteration 31381, loss = 0.52819372
Iteration 31382, loss = 0.52819375
Iteration 31383, loss = 0.52819371
Iteration 31384, loss = 0.52819367
Iteration 31385, loss = 0.52819368
Iteration 31386, loss = 0.52819362
Iteration 31387, loss = 0.52819361
Iteration 31388, loss = 0.52819363
Iteration 31389, loss = 0.52819359
Iteration 31390, loss = 0.52819353
Iteration 31391, loss = 0.52819350
Iteration 31392, los

Iteration 31600, loss = 0.52818700
Iteration 31601, loss = 0.52818697
Iteration 31602, loss = 0.52818691
Iteration 31603, loss = 0.52818687
Iteration 31604, loss = 0.52818685
Iteration 31605, loss = 0.52818682
Iteration 31606, loss = 0.52818678
Iteration 31607, loss = 0.52818674
Iteration 31608, loss = 0.52818669
Iteration 31609, loss = 0.52818666
Iteration 31610, loss = 0.52818664
Iteration 31611, loss = 0.52818660
Iteration 31612, loss = 0.52818659
Iteration 31613, loss = 0.52818652
Iteration 31614, loss = 0.52818646
Iteration 31615, loss = 0.52818651
Iteration 31616, loss = 0.52818637
Iteration 31617, loss = 0.52818632
Iteration 31618, loss = 0.52818629
Iteration 31619, loss = 0.52818628
Iteration 31620, loss = 0.52818622
Iteration 31621, loss = 0.52818617
Iteration 31622, loss = 0.52818612
Iteration 31623, loss = 0.52818607
Iteration 31624, loss = 0.52818605
Iteration 31625, loss = 0.52818608
Iteration 31626, loss = 0.52818601
Iteration 31627, loss = 0.52818601
Iteration 31628, los

Iteration 31847, loss = 0.52817991
Iteration 31848, loss = 0.52817987
Iteration 31849, loss = 0.52817986
Iteration 31850, loss = 0.52817982
Iteration 31851, loss = 0.52817980
Iteration 31852, loss = 0.52817980
Iteration 31853, loss = 0.52817971
Iteration 31854, loss = 0.52817968
Iteration 31855, loss = 0.52817968
Iteration 31856, loss = 0.52817968
Iteration 31857, loss = 0.52817963
Iteration 31858, loss = 0.52817963
Iteration 31859, loss = 0.52817962
Iteration 31860, loss = 0.52817959
Iteration 31861, loss = 0.52817958
Iteration 31862, loss = 0.52817958
Iteration 31863, loss = 0.52817959
Iteration 31864, loss = 0.52817955
Iteration 31865, loss = 0.52817953
Iteration 31866, loss = 0.52817952
Iteration 31867, loss = 0.52817952
Iteration 31868, loss = 0.52817944
Iteration 31869, loss = 0.52817938
Iteration 31870, loss = 0.52817935
Iteration 31871, loss = 0.52817933
Iteration 31872, loss = 0.52817927
Iteration 31873, loss = 0.52817924
Iteration 31874, loss = 0.52817919
Iteration 31875, los

In [4]:
print(nn.score(X_test,y_test))

0.746576701726533
